In [ ]:
HOME_DIR = "/content/drive/MyDrive/PhD research/LLM Privacy Policy"

In [ ]:
!pip -q install llama-index llama-index-embeddings-huggingface llama-index-llms-llama-cpp pypdf
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip -q install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 MB 12.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import time
from glob import glob
import pandas as pd

from transformers import LlamaForCausalLM, LlamaTokenizer
from llama_index.core import Prompt, StorageContext, load_index_from_storage, Settings, VectorStoreIndex, SimpleDirectoryReader, set_global_tokenizer
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.llama_cpp import LlamaCPP

from transformers import AutoTokenizer
from IPython.display import Markdown, display

# LLM Model

In [ ]:
text_embedding_model = 'thenlper/gte-base'  #Alt: thenlper/gte-base, jinaai/jina-embeddings-v2-base-en
llm_url = 'https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf'

# Load models and service context
embed_model = HuggingFaceEmbedding(model_name=text_embedding_model)
llm = LlamaCPP(model_url=llm_url, temperature=0.7, max_new_tokens=256, context_window=4096, generate_kwargs = {"stop": ["<s>", "[INST]", "[/INST]"]}, model_kwargs={"n_gpu_layers": -1}, verbose=True)
# service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model, chunk_size=512)
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

total size (MB): 4081.0


3892it [00:30, 127.17it/s]                         
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /tmp/llama_index/models/llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_load

In [ ]:
def extractQueryAnsFromPdf(query, pdf_path):
  filename_fn = lambda filename: {'file_name': os.path.basename(pdf_path)}
  loader = SimpleDirectoryReader(input_files=[pdf_path], file_metadata=filename_fn)
  documents = loader.load_data()

  # Indexing
  start_time = time.time()

  index = VectorStoreIndex.from_documents(documents, embed_model=embed_model, llm=llm)

  end_time = time.time()
  elapsed_time = end_time - start_time
  print(f"Elapsed indexing time: {elapsed_time:.2f} s")

  query_str = "What are the data the company collect?"
  query_engine = index.as_query_engine(similarity_top_k=1, llm=llm)
  # use this for testing
  vector_retriever = index.as_retriever(similarity_top_k=1)
  response = query_engine.query(query_str)
  return str(response)

In [ ]:
privacyPolicyPaths = glob("/content/drive/MyDrive/PhD research/LLM Privacy Policy/Codes/chatgpt privacy policy excel/downloaded policy pdf/*.pdf")
len(privacyPolicyPaths)

7

In [ ]:
dictAll = []
for pdf_path in privacyPolicyPaths:
  query = "What are the data the company collect?"
  response = extractQueryAnsFromPdf(query, pdf_path)
  print(response)

  dictAll.append({'file_name': os.path.basename(pdf_path), 'response': response})

Elapsed indexing time: 40.58 s


llama_perf_context_print:        load time =  106692.32 ms
llama_perf_context_print: prompt eval time =  106691.90 ms /   426 tokens (  250.45 ms per token,     3.99 tokens per second)
llama_perf_context_print:        eval time =   48837.85 ms /   133 runs   (  367.20 ms per token,     2.72 tokens per second)
llama_perf_context_print:       total time =  155588.46 ms /   559 tokens


 According to the context information, Google collects various types of data, including:
1. Activity on Google services, such as from your searches or places you label like home or work.
2. Information about things near your device, such as Wi-Fi access points, cell towers, and Bluetooth-enabled devices.
3. Information from publicly accessible sources, such as your name appearing in your local newspaper.
4. Information from trusted partners, such as directory services, marketing partners, and security partners.
5. Information from various technologies, including cookies, pixel tags, local storage, databases, and server logs.
Elapsed indexing time: 8.81 s


Llama.generate: 15 prefix-match hit, remaining 426 prompt tokens to eval
llama_perf_context_print:        load time =  106692.32 ms
llama_perf_context_print: prompt eval time =  110965.80 ms /   426 tokens (  260.48 ms per token,     3.84 tokens per second)
llama_perf_context_print:        eval time =   42119.58 ms /   106 runs   (  397.35 ms per token,     2.52 tokens per second)
llama_perf_context_print:       total time =  153135.69 ms /   532 tokens


 The company collects the following data:
• Information we collect directly from you when you provide it to us through our online services, such as when you sign up for an account, apply for a product or service, or use our online services. 
• Information we collect automatically as you use our online services, such as IP addresses, browser types, browser language, operating system, and pages visited. 
• Information from third parties, such as credit reporting agencies, to verify your identity and creditworthiness.
Elapsed indexing time: 24.30 s


...


The company collects the following data:
1. Information we collect directly from you:
This includes information you provide when you create an account, fill out a form, or participate in an activity on our Services, such as your name, email address, and password.
2. Information from your use of our Services:
This includes information about how you interact with our Services, such as the pages you visit, the features you use, and the actions you take.
3. Information from third parties:
This includes information we obtain from other companies, organizations, or individuals, such as demographic information, interest data, or other information that may be relevant to our business.
4. Context information:
This includes information about how you use our Services, such as the pages you visit, the features you use, and the actions you take.
5. Analytics and Improvement:
This includes information we collect to better understand how users access and use the Services, and our other products and 

Llama.generate: 15 prefix-match hit, remaining 650 prompt tokens to eval
llama_perf_context_print:        load time =  106692.32 ms
llama_perf_context_print: prompt eval time =  166784.91 ms /   650 tokens (  256.59 ms per token,     3.90 tokens per second)
llama_perf_context_print:        eval time =   51247.22 ms /   129 runs   (  397.27 ms per token,     2.52 tokens per second)
llama_perf_context_print:       total time =  218093.86 ms /   779 tokens



Based on the context information provided, Wells Fargo collects personal information from customers when they open an account, apply for a loan, or seek investment advice. They also collect personal information from others, such as credit bureaus, affiliates, and other companies.
The company collects the following data:
• Personal information when opening an account or making deposits or withdrawals from accounts
• Information about creditworthiness for affiliates’ everyday business purposes
• Information to market to you from affiliates and nonaffiliates
• Information about your creditworthiness from credit bureaus


Elapsed indexing time: 13.03 s


Llama.generate: 15 prefix-match hit, remaining 557 prompt tokens to eval
llama_perf_context_print:        load time =  106692.32 ms
llama_perf_context_print: prompt eval time =  142906.36 ms /   557 tokens (  256.56 ms per token,     3.90 tokens per second)
llama_perf_context_print:        eval time =  110928.02 ms /   255 runs   (  435.01 ms per token,     2.30 tokens per second)
llama_perf_context_print:       total time =  253985.78 ms /   812 tokens


 Based on the context information, the company collects various types of data, including:
1. Personal information: The company collects personal information such as name, address, date of birth, and Social Security number.
2. Health information: The company collects health information, including medical history, diagnoses, procedures, and treatment plans.
3. Claims information: The company collects claims information, including details about medical services rendered, claims submitted, and payments made.
4. Payment information: The company collects payment information, including payment amounts, payment dates, and payment providers.
5. Member ID: The company collects member ID numbers to identify individuals in their database.
6. Policy information: The company collects policy information, including policy numbers, effective dates, and expiration dates.
7. Subsidiaries and affiliates: The company collects information about its subsidiaries and affiliates, including their names, address

In [ ]:
dictAll

[{'file_name': 'Alphabet_(Google).pdf',
  'response': ' According to the context information, Google collects various types of data, including:\n1. Activity on Google services, such as from your searches or places you label like home or work.\n2. Information about things near your device, such as Wi-Fi access points, cell towers, and Bluetooth-enabled devices.\n3. Information from publicly accessible sources, such as your name appearing in your local newspaper.\n4. Information from trusted partners, such as directory services, marketing partners, and security partners.\n5. Information from various technologies, including cookies, pixel tags, local storage, databases, and server logs.'},
 {'file_name': 'JPMorgan_Chase_&_Co..pdf',
  'response': ' The company collects the following data:\n• Information we collect directly from you when you provide it to us through our online services, such as when you sign up for an account, apply for a product or service, or use our online services. \n• 

In [ ]:
# Convert dictionary to DataFrame
df = pd.DataFrame(dictAll)

# Save to CSV
df.to_excel("/content/drive/MyDrive/PhD research/LLM Privacy Policy/Codes/chatgptOnes.xlsx", index=False)

In [ ]:
df

,file_name,response
0,Alphabet_(Google).pdf,"According to the context information, Google ..."
1,JPMorgan_Chase_&_Co..pdf,The company collects the following data:\n• I...
2,The_Cigna_Group.pdf,\nThe company collects the following data:\n1....
3,Bank_of_America_Corp..pdf,\nThe company collects the following types of ...
4,Cigna_Healthcare.pdf,\nThe company collects the following data:\n1....
5,Wells_Fargo_&_Co..pdf,"\nBased on the context information provided, W..."
6,Humana_Inc..pdf,"Based on the context information, the company..."
